<a href="https://colab.research.google.com/github/xloem/apifudge/blob/main/test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip3 install adapter-transformers accelerate bitsandbytes
!git clone https://github.com/xloem/apifudge
!nvidia-smi
!ln -vsf apifudge/* .

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 54.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 191.5/191.5 KB 24.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.3/76.3 MB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 32.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.4/182.4 KB 21.6 MB/s eta 0:00:00
Cloning into 'apifudge'...
remote: Enumerating objects: 36, done.
remote: Counting objects: 100% (36/36), done.
remote: Compressing objects: 100% (24/24), done.
remote: Total 36 (delta 12), reused 24 (delta 7), pack-reused 0
Unpacking objects: 100% (36/36), done.
Sat Jan  7 18:05:51 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+-----------

In [3]:
# use drive to cache downloads
from google.colab import drive
drive.mount('/content/drive')
!mkdir -p /content/drive/MyDrive/colab/.cache
!rm -rf $HOME/.cache
!ln -s /content/drive/MyDrive/colab/.cache $HOME/.cache

Mounted at /content/drive


In [4]:
import apifudge1 as af1
import transformers
npapi = af1.API('numpy')
npapi.set_example(
'''Construct an array B that repeats array Y 4 times along the last dimension.''',
'''B = np.tile(Y, 4)'''
)
npapi.set_example(
'''Swap the last two axes of an array D.''',
'''D = D.swapaxes(-2, -1)'''
)
npapi.set_example(
'''Sort an array along the third axis.''',
'''array.sort(3)'''
)
for name, text in npapi.get_examples().items():
    print(f'{name}: {repr(text)}')
print('loading ...')
pipeline = transformers.pipeline(
    'text-generation', 'bigscience/bloomz-3b', 
    model_kwargs=dict(device_map='auto'))
generator = af1.Generator(pipeline)
print('Augmenting ...')
for prompt, result in generator.augment(npapi):
  print(f'{prompt}: {repr(result)}')
print('Inferring:')
prompt = 'Reverse an array C.'
output = generator(npapi, prompt)
print(f'{prompt}: {repr(output)}')

Sort an array along the third axis.: 'array.sort(3)'
Swap the last two axes of an array D.: 'D = D.swapaxes(-2, -1)'
Construct an array B that repeats array Y 4 times along the last dimension.: 'B = np.tile(Y, 4)'
loading ...


Augmenting ...
Apply a function to each element of an array D.: 'D = D.map(lambda x: x**2)'
Apply a function to each element of an array D.: 'D = D.map(lambda x: x**3)'
Apply a function to each element of an array D.: 'D = D.map(lambda x: x**4)'
Apply a function to each element of an array D.: 'D = D.map(lambda x: x**5)'
Apply a function to each element of an array D.: 'D = D.map(lambda x: x**6)'
Apply a function to each element of an array D.: 'D = D.map(lambda x: x**7)'
Apply a function to each element of an array D.: 'D = D.map(lambda x: x**8)'
Apply a function to each element of an array D.: 'D = D.map(lambda x: x**9)'
Apply a function to each element of an array D.: 'D = D.map(lambda x: x**10)'
Apply a function to each element of an array D.: 'D = D.map(lambda x: x**11)'
Apply a function to each element of an array D.: 'D = D.map(lambda x: x**12)'
Apply a function to each element of an array D.: 'D = D.map(lambda x: x**13)'


KeyboardInterrupt: ignored